In [1]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax
from jax.tree_util import tree_flatten, tree_unflatten
from functools import partial
import os
import requests

import numpy as np
import os
from tqdm import tqdm
import time
import yaml

import syrkis
from src.model import init_fn, apply_fn, loss_fn, block_fn, generate_fn
from src.data import get_batches, encode, decode
from src.train import train_fn

import syrkis

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-14 23:21:05.722066: W pjrt_plugin/src/mps_client.cc:534] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


In [2]:
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
rng = jax.random.PRNGKey(0)
rng, key = jax.random.split(rng)
train_batches = get_batches(rng, 'train')
eval_batches = get_batches(rng, 'test')

In [4]:
rng, key = jax.random.split(rng)
params = init_fn(key, config)
print('n_params:', syrkis.training.n_params(params))
opt = optax.adamw(config['lr'])
opt_state = opt.init(params)

n_params: 85952


In [5]:
opt_state, params = train_fn(key, opt, opt_state, params, config, train_batches, eval_batches)
syrkis.training.save_params(params)
# params = syrkis.training.load_params()

In [6]:
seed = 'On latitudes this low'
rng, key = jax.random.split(rng)
idx = jnp.array(encode(seed))[None, :]
print(decode(generate_fn(key, params, idx, config['block_size'], length=100)[0].tolist()))

 22%|██▏       | 22/100 [00:17<01:02,  1.25it/s]


KeyboardInterrupt: 